## Training the Model

First, train a linear regression model that takes two hyperparameters: alpha and l1_ratio.

This example uses the familiar pandas, numpy, and sklearn APIs to create a simple machine learning model. The MLflow tracking APIs log information about each training run like hyperparameters (alpha and l1_ratio) used to train the model, and metrics (root mean square error, mean absolute error, and r2) used to evaluate the model. The example also serializes the model in a format that MLflow knows how to deploy.

Each time you run the example, MLflow logs information about your experiment runs in the directory mlruns.

You can run the example through the .py script using the following command.
* python train.py <alpha> <l1_ratio>

Or you can also use the notebook code below that does the same thing

In [2]:
# Wine Quality Sample

def train(in_alpha, in_l1_ratio):
    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet
    # import mlflow
    # import mlflow.sklearn

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    data = pd.read_csv(csv_url, sep=';')

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Execute ElasticNet
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    # Evaluate Metrics
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    # Print out metrics
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

In [3]:
# Run the above training code with different hyperparameters (9 runs)
alphas = [0.25, 0.5, 0.75]
l1_ratios = [0.25, 0.5, 0.75]
for i in alphas:
    for j in l1_ratios:
        train(i,j)

Elasticnet model (alpha=0.250000, l1_ratio=0.250000):
  RMSE: 0.7380489682487518
  MAE: 0.5690312554727687
  R2: 0.22820122626467798
Elasticnet model (alpha=0.250000, l1_ratio=0.500000):
  RMSE: 0.748930783857188
  MAE: 0.5806946169417598
  R2: 0.20527460024945354
Elasticnet model (alpha=0.250000, l1_ratio=0.750000):
  RMSE: 0.7662476663327954
  MAE: 0.5985976516559472
  R2: 0.1680982095420568
Elasticnet model (alpha=0.500000, l1_ratio=0.250000):
  RMSE: 0.7596554775612442
  MAE: 0.5913132541174235
  R2: 0.18235068599935977
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614
Elasticnet model (alpha=0.500000, l1_ratio=0.750000):
  RMSE: 0.8318658159940802
  MAE: 0.6651040854928951
  R2: 0.019516509058132292
Elasticnet model (alpha=0.750000, l1_ratio=0.250000):
  RMSE: 0.7837307525653582
  MAE: 0.6165474987409884
  R2: 0.1297029612600864
Elasticnet model (alpha=0.750000, l1_ratio=0.500000):
  RMSE: 0.8318702